No quotation marks in the prompt. Enter your prompt and the play button for that cell. Press play on the pip install cell before doing prompts.

In [ ]:
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Load personas.csv

In [ ]:
import pandas as pd
from tabulate import tabulate

from IPython.display import display, HTML
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

dirpath_personas = "/content/drive/YOUR_PATH" #@param {type: "string"}
personas_path = dirpath_personas + 'personas.csv' #@param {type: "string"}
ndataf = pd.DataFrame()
pdataf = pd.DataFrame()
pdataf = df=pd.read_csv(personas_path)
max_peeps = pdataf.shape[0]
max_peeps
pdataf.shape
#list(pdataf.columns) #deal with space char prefix column names
# pretty_print(pdataf)
# print(tabulate([list(row) for row in df.values], headers=list(df.columns)))

In [ ]:
import openai
import pandas as pd
import difflib
from datetime import datetime
import time

# @title Setup Parameters
# Set your OpenAI API key
openai.api_key = 'YOUR_KEY'

TEMPERATURE = 0.2
FREQUENCY_PENALTY = 0.0

# Define the maximum number of retries and initial wait time
max_retries = 5
wait_time = 1  # in seconds

system_text = f"""
BACKGROUND TEXT FOR THE RECENT WORLD EVENTS, YOUR CAMPAIGN DESCRIPTION, ....
"""

insert_prompt = "THE ACTUAL PROMPT"  #@param {type: "string"}

number_of_results = 2  #@param {type: "number"}
pick_model = "gpt-3.5-turbo-16k"  #@param ['gpt-3.5-turbo-16k', 'gpt-3.5-turbo', 'gpt-4']

this_platform = "Twitter" #@param {type: "string"}
response_max_tokens=200 #@param {type: "number"}

if number_of_results > max_peeps:
   number_of_results = max_peeps

ndataf = pdataf.iloc[:number_of_results,:16].copy() # truncate
ndataf.shape
# pretty_print(ndataf)


# Functions

In [ ]:
def build_author(me): #can be modified - is used to build the prompt
  about_me =  " I am from " + me[' location'] \
  + ". my twitter bio is: " + me[' twitter bio'] \
  + ". I write with a tone and voice of " + me[' tone and voice'] \
  + ". My geographical and cultural context is: " + me[' geographical and cultural context'] + "." \
  + " My usual political perspective is based on" + me[' beliefs and values'] + "." \
  + " My writing style is:" + me[' writing style'] +"."
  return about_me


def get_completion(messages, model=pick_model, temperature=TEMPERATURE,frequency_penalty=FREQUENCY_PENALTY):
#    try:
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        frequency_penalty=frequency_penalty,
    )
#        break  # if the request was successful, break the retry loop -- B's error recovery
#    except openai.error.ServiceUnavailableError:
#            if i < max_retries - 1:  # no need to wait on the last attempt
#                time.sleep(wait_time)
#                wait_time *= 2  # double the wait time for the next attempt
#            else:
#                raise  # re-raise the last exception if all retries failed
    return response.choices[0].message["content"].strip()


def build_prompt_get_completion(persona, system_text, insert_prompt):

    mystory = build_author(persona)
    prompt1 = f"""
    Write a single tweet using the perspective and voice without being overly personal of {mystory} .
    This tweet MUST BE no more than {response_max_tokens} tokens in length but can be shorter.
    The tweet must be based on the content of the folowing text: {insert_prompt}.
    The response must be in English language.
    Do not begin the tweet with 'As of'
"""
    messages =  [
        {'role':'system', 'content':system_text},
        {'role':'user', 'content': prompt1 } ]
    response = get_completion(messages)
    return response

# Time to build all the responses

In [ ]:

ndataf['response1'] = ndataf.apply(lambda x: build_prompt_get_completion(x, system_text, insert_prompt), axis=1)

savedf = pd.DataFrame(ndataf, columns=[' twitter handle', 'response1'])


In [ ]:
pretty_print(savedf.head(40))

# Save Results

In [ ]:

# Save to a CSV file
abrevstr = ''.join([ s[0] for s in insert_prompt.split() ])
date_string = datetime.now().strftime("%m-%d-%s")


#df.to_csv(f'/content/out.csv', index=False)
savedf.to_csv(f'/content/{date_string}_{abrevstr}.csv', index=False)

In [ ]:
!pwd
!ls